<a href="https://colab.research.google.com/github/kimhkh/movieRecommendationSystem/blob/Kam_branch/movie_recommendation_system_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install surprise

     |████████████████████████████████| 11.8MB 225kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617600 sha256=4cffe15a333f99584802b49e358b88b3466175bee4f657d198f6827953956262
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
pip install pytorch_lightning


     |████████████████████████████████| 808kB 25.8MB/s 
     |████████████████████████████████| 645kB 44.1MB/s 
     |████████████████████████████████| 276kB 54.5MB/s 
     |████████████████████████████████| 829kB 51.5MB/s 
     |████████████████████████████████| 112kB 48.5MB/s 
     |████████████████████████████████| 1.3MB 48.1MB/s 
     |████████████████████████████████| 143kB 57.6MB/s 
     |████████████████████████████████| 296kB 47.9MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=956faf0c6b1026e95c6467afa9d6fcfb9e9489510e2029ff04c9d93287e337c1
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [3]:
from surprise import SVD, accuracy, SVDpp
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np
import os
from surprise import Reader, Dataset
import seaborn as sns
import matplotlib.pyplot as plt
import time
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset as d
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from datetime import datetime

In [4]:
df_rating = pd.read_csv('/content/drive/MyDrive/movielens1B/ratings.csv')

In [5]:
np.random.seed(123)
rand_userIds = np.random.choice(df_rating['userId'].unique(), 
                                size=int(len(df_rating['userId'].unique())*0.05), 
                                replace=False)

df_rating = df_rating.loc[df_rating['userId'].isin(rand_userIds)]

print('There are {} rows of data from {} users'.format(len(df_rating), len(df_rating['userId'].unique())))
df_rating["timestamp"] = df_rating.timestamp.apply(lambda x: datetime.fromtimestamp(x / 1e3))

There are 1355305 rows of data from 14161 users


In [6]:
df_rating.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1970-01-15 13:04:37.221
1,1,481,3.5,1970-01-15 13:04:37.456
2,1,1091,1.5,1970-01-15 13:04:37.471
3,1,1257,4.5,1970-01-15 13:04:37.460
4,1,1449,4.5,1970-01-15 13:04:37.264


In [ ]:
#Loading the df_rating from a pandas dataframe using load_from_df() method and reader object

reader = Reader(rating_scale=(0.5,5))
data = Dataset.load_from_df(df_rating[['userId', 'movieId', 'rating']], reader)
trainset, testset = trainset, testset = train_test_split(data, test_size=0.20)

In [ ]:
#Using the SVD algorithm
import time
start = time.time()
algo = SVD()
#Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)
end = time.time()
time = end-start
print(time)

RMSE: 0.8126
293.9931712150574


In [ ]:
import time
start1 = time.time()
cross_validate(algo, data, measures = ['RMSE'],cv=5,verbose=True) 
end1 = time.time()
time = end1-start1
print(time)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8130  0.8123  0.8125  0.8128  0.8125  0.8126  0.0003  
Fit time          277.49  281.42  280.24  286.36  276.89  280.48  3.38    
Test time         16.05   15.94   15.67   15.53   15.65   15.77   0.19    
1569.7276475429535


In [ ]:
import time
start = time.time()
algo= SVDpp()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)
end = time.time()
time = end-start
print(time)

KeyboardInterrupt: ignored

In [ ]:
import time
start1 = time.time()
cross_validate(algo, data, measures = ['RMSE'],cv=5,verbose=True) 
end1 = time.time()
time = end1-start1
print(time)

In [7]:
df_rating['rank_latest'] = df_rating.groupby(['userId'])['timestamp'] \
                                .rank(method='first', ascending=False)
df_rating

,userId,movieId,rating,timestamp,rank_latest
0,1,307,3.5,1970-01-15 13:04:37.221,15.0
1,1,481,3.5,1970-01-15 13:04:37.456,8.0
2,1,1091,1.5,1970-01-15 13:04:37.471,5.0
3,1,1257,4.5,1970-01-15 13:04:37.460,7.0
4,1,1449,4.5,1970-01-15 13:04:37.264,10.0
...,...,...,...,...,...
27753326,283227,3006,5.0,1970-01-16 14:03:53.534,15.0
27753327,283227,3256,5.0,1970-01-16 14:03:53.536,14.0
27753328,283227,4002,4.5,1970-01-16 14:03:53.623,3.0
27753329,283227,4321,4.5,1970-01-16 14:03:53.626,2.0


In [8]:
#Using earlier review for trainning, and using latest review for testing
train_ratings = df_rating[df_rating['rank_latest'] != 1]
test_ratings = df_rating[df_rating['rank_latest'] == 1]
train_ratings

,userId,movieId,rating,timestamp,rank_latest
0,1,307,3.5,1970-01-15 13:04:37.221,15.0
1,1,481,3.5,1970-01-15 13:04:37.456,8.0
2,1,1091,1.5,1970-01-15 13:04:37.471,5.0
3,1,1257,4.5,1970-01-15 13:04:37.460,7.0
4,1,1449,4.5,1970-01-15 13:04:37.264,10.0
...,...,...,...,...,...
27753325,283227,2881,4.0,1970-01-16 14:03:53.588,7.0
27753326,283227,3006,5.0,1970-01-16 14:03:53.534,15.0
27753327,283227,3256,5.0,1970-01-16 14:03:53.536,14.0
27753328,283227,4002,4.5,1970-01-16 14:03:53.623,3.0


In [9]:
#Converting the dataset into an implicit feedback dataset
#Binarize the ratings to 1 means the user has interacted with the movie
train_ratings.loc[:, 'rating'] = 1
train_ratings.sample(10)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,userId,movieId,rating,timestamp,rank_latest
6220559,63847,634,1,1970-01-10 20:13:34.790,2.0
4716387,48402,1285,1,1970-01-16 16:05:45.322,172.0
9982966,102830,337,1,1970-01-10 15:25:57.699,8.0
17935661,183317,1772,1,1970-01-11 21:26:58.478,86.0
20046869,204398,3556,1,1970-01-12 13:08:17.497,8.0
10874741,111777,72167,1,1970-01-17 11:55:32.643,20.0
12592884,128959,114180,1,1970-01-17 22:19:03.529,1241.0
9854485,101578,55280,1,1970-01-15 02:25:33.751,182.0
17620602,180179,142488,1,1970-01-17 22:00:02.730,26.0
785104,7932,89087,1,1970-01-17 14:32:51.774,2054.0


In [10]:
# Generate negative samples to train our models
# Get a list of all movie IDs
all_movieIds = df_rating['movieId'].unique()

# Placeholders that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each user has interaction with
user_item_set = set(zip(train_ratings['userId'], train_ratings['movieId']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in tqdm(user_item_set):
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_movieIds) 
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_movieIds)
        users.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative

100%|██████████| 1341144/1341144 [00:27<00:00, 48904.00it/s]


In [11]:
class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training
    
    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds
    
    """

    def __init__(self, ratings, all_movieIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_movieIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['userId'], ratings['movieId']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_movieIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_movieIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

In [12]:
class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            all_movieIds (list): List containing all movieIds (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=512, num_workers=1)

In [13]:
num_users = df_rating['userId'].max()+1
num_items = df_rating['movieId'].max()+1

all_movieIds = df_rating['movieId'].unique()

model = NCF(num_users, num_items, train_ratings, all_movieIds)

In [14]:
import time
start = time.time()
trainer = pl.Trainer(max_epochs=3, gpus=1, reload_dataloaders_every_epoch=True,
                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)

trainer.fit(model)
end = time.time()
time = end-start

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 2.3 M 
1 | item_embedding | Embedding | 1.6 M 
2 | fc1            | Linear    | 1.1 K 
3 | fc2            | Linear    | 2.1 K 
4 | output         | Linear    | 33    
---------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.278    Total estimated model params size (MB)


In [ ]:
print(time)

In [52]:
# User-item pairs for testing

test_user_item_set = set(zip(test_ratings['userId'], test_ratings['movieId']))

# Dict of all items that are interacted with by each user
user_interacted_items = df_rating.groupby('userId')['movieId'].apply(list).to_dict()

hits = []
for (u,i) in tqdm(test_user_item_set):
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_movieIds) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The accuracy of the recommended top 10 items that the user will interact is {:.2f}".format(np.average(hits)))

100%|██████████| 14161/14161 [01:49<00:00, 129.75it/s]

The accuracy of the recommended top 10 items that the user will interact is 1.00
